# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [14]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [15]:
#### Creating list of filepaths to process original event csv data files

In [16]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root, '*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [17]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding='utf8', newline='') as csvfile:
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        next(csvreader)

        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

        # uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new2.csv', 'w', encoding='utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
                     'level', 'location', 'sessionId', 'song', 'userId'])
    for row in full_data_rows_list:
        if row[0] == '':
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [18]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding='utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="../resources/img/img.png">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [19]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1) docker run -p 9042:9042 --rm --name cassandra -d cassandra:3.11

from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])  #['127.0.0.1']

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [20]:
try:
    session.execute(
        """CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )

except Exception as e:
    print(e)

#### Set Keyspace

In [21]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [22]:
drop_table_query = "DROP TABLE IF EXISTS {table_name}"
create_table_query = "CREATE TABLE IF NOT EXISTS {table_name} ({atbs})"
create_primary_key = "PRIMARY KEY ({primary_keys})"
insert_query = "INSERT INTO {table_name} ({atbs}) VALUES ({values})"
select_query = "SELECT {atbs} FROM {table_name} {where} {order};"
create_index_query = "CREATE INDEX {index_name} ON {table_name} ({col})"
where = "WHERE {condition}"
order = "ORDER BY {order}"

## Table song_details

In [23]:
try:
    new_table = 'song_details'
    session.execute(drop_table_query.format(table_name=new_table))
    session.execute(
        create_table_query.format(
            table_name=new_table,
            atbs=f"""
                sessionId bigint,
                itemInSession bigint,
                artist text,
                song text,
                length float,
                {create_primary_key.format(primary_keys="sessionId, itemInSession")} 
            """
        )  #query (1) need query by itemInSession, sessionId 
    )
except Exception as e:
    print(e)




## Table user_details

In [24]:
try:
    new_table = 'song_history'
    session.execute(drop_table_query.format(table_name=new_table))
    session.execute(
        create_table_query.format(
            table_name=new_table,
            atbs=f"""
                userId bigint,
                sessionId bigint,
                itemInSession bigint,
                firstName text,
                lastName text,
                artist text,
                song text,
                {create_primary_key.format(primary_keys="(userId, sessionId), itemInSession")} 
            """
        )  #query (2) need query by (userId, sessionId) and order by itemInSession
    )
except Exception as e:
    print(e)

In [25]:
try:
    new_table = 'user_history'
    session.execute(drop_table_query.format(table_name=new_table))
    session.execute(
        create_table_query.format(
            table_name=new_table,
            atbs=f"""
                userId bigint,
                song text,
                firstName text,
                lastName text,
                {create_primary_key.format(primary_keys="song, userId")} 
            """
        )  
    )
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [26]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    reader = csv.reader(f)
    next(reader)
    for line in reader:
        session.execute(
            insert_query.format(
                table_name="song_details",
                atbs="sessionId, itemInSession, artist, song, length",
                values="%s, %s, %s, %s, %s"),
            (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))
        )
        session.execute(
            insert_query.format(
                table_name="song_history",
                atbs="userId,sessionId, itemInSession,firstName, lastName, artist, song",
                values="%s, %s, %s, %s,%s, %s, %s"),
            (int(line[10]),int(line[8]), int(line[3]), line[1], line[4], line[0], line[9])
        )
        session.execute(
            insert_query.format(
                table_name="user_history",
                atbs="userId,song, firstName, lastName",
                values="%s, %s, %s, %s"),
            (int(line[10]), line[9], line[1], line[4])
        )

#### Do a SELECT to verify that the data have been inserted into each table

In [27]:
try:
    song_details = pd.DataFrame(list(session.execute(select_query.format(atbs='*',
                                                                       table_name='song_details',
                                                                       where="",
                                                                       order=""))))
    print(song_details)
    song_history = pd.DataFrame(list(session.execute(select_query.format(atbs='*',
                                                                    table_name='song_history',
                                                                    where="",
                                                                    order=""))))
    print(song_history)
    user_history = pd.DataFrame(list(session.execute(select_query.format(atbs='*',
                                                                    table_name='user_history',
                                                                    where="",
                                                                    order=""))))
    print(user_history)
except Exception as e:
    print(e)


In [28]:
## Give me the artist, song title and song's length in the music app history that was heard during 
# sessionId = 338, and itemInSession = 4

query1 = select_query.format(atbs="artist, song, length", table_name='song_details',
                             where=where.format(condition="sessionId =338 and itemInSession =4"),
                             order=""
                             )

try:
    df_output = pd.DataFrame(list(session.execute(query1)))
    print(df_output)
except Exception as e:
    print(e)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [29]:
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query_session = select_query.format(atbs="firstName,lastName, artist, song",
                                    table_name='song_history',
                                    where=where.format(
                                        condition="userId = 10 and sessionId = 182"),
                                    order=order.format(order="itemInSession"))

try:
    df_output = pd.DataFrame(list(session.execute(query_session)))
    print(df_output)
except Exception as e:
    print(e)


In [30]:
##Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


query_session = select_query.format(atbs="firstName, lastName",
                                    table_name='user_history',
                                    where=where.format(condition="song = 'All Hands Against His Own'"), order="")

try:
    df_output = pd.DataFrame(list(session.execute(query_session)))
    print(df_output)
except Exception as e:
    print(e)


### Drop the tables before closing out the sessions

In [31]:
## TO-DO: Drop the table before closing out the sessions

In [32]:
try:
    session.execute(drop_table_query.format(table_name="session"))
    session.execute(drop_table_query.format(table_name="user"))
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()